In [1]:
import dask
import pandas as pd
import dask.dataframe as dd

# Alcock

In [3]:
metadata = pd.read_csv('./raw_data/MACHO/Variables/Alcock2001/Final_Catalog_Alcock2001.dat')
metadata.sample()

,ID,Class,rPer,bPer,Path_B,Path_R
20182,82.8771.967,RRab,0.59575,0.59576,./LCs/B/82.8771.967.dat,./LCs/R/82.8771.967.dat


In [7]:
%%time
max_size = 100
dataframes = []
tot_size = 0.
shard = 0
nobs = []
for k, row in metadata.iterrows():

    lc = pd.read_csv('./raw_data/MACHO/Variables/Alcock2001/LCs/R/'+row['ID']+'.dat', 
                 delim_whitespace=False)
  
    lc['oid'] = [row['ID']]*lc.shape[0]
    lc['band'] = [1]*lc.shape[0]
    lc = lc.rename(columns={'err':'std'})
    lc = lc[['oid', 'mjd', 'mag', 'std', 'band']]

    nobs.append(lc.shape[0])
    dataframes.append(lc)
    memory = (lc.memory_usage()*1e-6).sum()
    tot_size+=memory
    
    if tot_size > 100 or k == metadata.shape[0]-1:
        df = pd.concat(dataframes)
        df.to_csv('/tf/astromer/data/raw_data/alcock/phot/{}.csv'.format(shard), index=False)
        shard+=1
        dataframes = []
        tot_size = 0.

CPU times: user 2min 16s, sys: 896 ms, total: 2min 17s
Wall time: 2min 18s


In [ ]:
metadata['nobs'] = nobs
metadata = metadata[['ID', 'Path_R', 'Class', 'nobs']]
metadata.columns = ['oid', 'path', 'class', 'nobs']
metadata.sample(3)

In [13]:
metadata.to_csv('/tf/astromer/data/raw_data/alcock/metadata.csv', 
                index=False)